In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
# Add the fourier_learning_ibm package to the path
import sys, pprint

sys.path.append("/home/jovyan/fourier_learning_ibm/")
pprint.pprint(sys.path)

['/home/jovyan',
 '/opt/conda/lib/python311.zip',
 '/opt/conda/lib/python3.11',
 '/opt/conda/lib/python3.11/lib-dynload',
 '',
 '/opt/conda/lib/python3.11/site-packages',
 '/home/jovyan/fourier_learning_ibm/']


In [3]:
import numpy as np
import scipy
import matplotlib.pyplot as plt
import pandas as pd
import os
from heisenberg_graph import (
    HeisenbergModel,
    get_n_steps,
    get_graph,
    get_positions,
    get_initial_layout,
    get_prob0,
    extract_probs,
)
from qiskit_ibm_runtime import QiskitRuntimeService, SamplerV2 as Sampler, Batch
from qiskit_aer import AerSimulator
from qiskit_aer.noise import NoiseModel
import mthree
import pickle
import networkx as nx
from datetime import datetime, timezone
import json
import math

In [4]:
# Option1: Use IBM Quantum backend.
# Set up the Qiskit Runtime service (this is a one-time setup)
# QiskitRuntimeService.save_account(
#     token="YOUR_API_TOKEN",
#     channel="ibm_quantum",
# )

# Load saved credentials
service = QiskitRuntimeService()
# backend_qpu = service.least_busy(simulator=False, interactional=True)
backend_qpu = service.backend("ibm_marrakesh")

# Option2: Use local AerSimulator as the backend.
backend_sim = AerSimulator()

noise_model = NoiseModel.from_backend(backend_qpu)
backend_sim_noisy = AerSimulator(noise_model=noise_model)

print(f"Using backend QPU: {backend_qpu}")
print(f"Using backend simulator: {backend_sim}")
print(f"Using backend noisy simulator: {backend_sim_noisy}")

Using backend QPU: <IBMBackend('ibm_marrakesh')>
Using backend simulator: AerSimulator('aer_simulator')
Using backend noisy simulator: AerSimulator('aer_simulator'
             noise_model=<NoiseModel on ['cz', 'id', 'sx', 'x', 'measure', 'reset']>)


In [ ]:
# Only for using Docker container
%cd fourier_learning_ibm/

/home/jovyan/fourier_learning_ibm


In [6]:
# Load parameters
# 'path' is the path created by the 'data_generate.ipynb'
path = f"data/2024-12-19T04:50Z"

# Load the graphs as a binary file
with open(f"{path}/params_object.pkl", "rb") as f:
    params_object = pickle.load(f)

graphs = params_object["graphs"]
all_Js = params_object["Js"]
all_expected_values = params_object["expected_values"]

# Load the parameters as a JSON file
with open(f"{path}/params_text.json", "r") as f:
    params = json.load(f)

n_samples = params["n_samples"]
n_qubits = params["n_qubits"]
graph_type = params["graph_type"]
backend_qpu_name = params["backend_qpu_name"]
beta = params["beta"]
C = params["C"]
n_features = params["n_features"]
times = params["times"]

for key, value in params.items():
    print(f"{key}: {value}")

n_samples: 70
n_qubits: 12
graph_type: line
backend_qpu_name: ibm_marrakesh
beta: 1
C: 0.39777364186486336
n_features: 10
times: [0.0, 7.897940745548682, 15.795881491097363, 23.693822236646046, 31.591762982194727, 39.489703727743404, 47.38764447329209, 55.28558521884077, 63.18352596438945, 71.08146670993813]


# Create Fourier feature

## Trotter simulation (Exact)

In [7]:
# Exact simulation
data = []
probs_phase0_exact = {}
probs_phase1_exact = {}
probs_phase2_exact = {}
probs_phase3_exact = {}

# Generate the dataset with Fourier features
for i in range(n_samples):
    print(f"Calculating features for sample {i}/{n_samples}")
    Js = all_Js[i]
    lambda_ref = np.sum(Js)  # Reference eigenvalue.
    heisenberg_exact = HeisenbergModel(n_qubits, graphs[i])
    features_exact = []

    probs_phase0_exact[f"sample{i}"] = {}
    probs_phase1_exact[f"sample{i}"] = {}
    probs_phase2_exact[f"sample{i}"] = {}
    probs_phase3_exact[f"sample{i}"] = {}

    # Compute the Fourier features for different times
    for k in range(n_features):
        prob_phase0 = heisenberg_exact.exact_simulation(times[k], phase=0)
        prob_phase1 = heisenberg_exact.exact_simulation(times[k], phase=1)
        prob_phase2 = heisenberg_exact.exact_simulation(times[k], phase=2)
        prob_phase3 = heisenberg_exact.exact_simulation(times[k], phase=3)

        probs_phase0_exact[f"sample{i}"][f"f_{k}"] = prob_phase0
        probs_phase1_exact[f"sample{i}"][f"f_{k}"] = prob_phase1
        probs_phase2_exact[f"sample{i}"][f"f_{k}"] = prob_phase2
        probs_phase3_exact[f"sample{i}"][f"f_{k}"] = prob_phase3

        inner_product = np.exp(-1j * lambda_ref * times[k]) * (
            (prob_phase0 - prob_phase1) + 1j * (prob_phase2 - prob_phase3)
        )

        features_exact.append(inner_product.real)
        if k != 0:
            features_exact.append(inner_product.imag)
    data.append([i, *features_exact, all_expected_values[i]])

# Create column names for the DataFrame
columns = []
columns.append("sample_id")
for k in range(n_features):
    columns.append(f"f_{k} Re")
    if k != 0:
        columns.append(f"f_{k} Im")
columns.append("expected_value")

# Convert to a DataFrame
data_exact_df = pd.DataFrame(data, columns=columns)
display(data_exact_df)

# Save the exact data
data_exact_df.to_json(f"{path}/data_exact_df.json")
with open(f"{path}/probs_phase0_exact.json", "w") as f:
    json.dump(probs_phase0_exact, f)

with open(f"{path}/probs_phase1_exact.json", "w") as f:
    json.dump(probs_phase1_exact, f)

with open(f"{path}/probs_phase2_exact.json", "w") as f:
    json.dump(probs_phase2_exact, f)

with open(f"{path}/probs_phase3_exact.json", "w") as f:
    json.dump(probs_phase3_exact, f)

Calculating features for sample 0/70


/opt/conda/lib/python3.11/site-packages/scipy/sparse/linalg/_dsolve/linsolve.py:412: SparseEfficiencyWarning: splu converted its input to CSC format
  warn('splu converted its input to CSC format', SparseEfficiencyWarning)
/opt/conda/lib/python3.11/site-packages/scipy/sparse/linalg/_dsolve/linsolve.py:302: SparseEfficiencyWarning: spsolve is more efficient when sparse b is in the CSC matrix format
  warn('spsolve is more efficient when sparse b '


KeyboardInterrupt: 

## Trotter simulation (Simulator, Noise free)

In [8]:
# Create circuits
# 1 jobs - 1 sample
circuits_phase0 = {}
circuits_phase1 = {}
circuits_phase2 = {}
circuits_phase3 = {}
exec_circuits_phase0 = {}
exec_circuits_phase1 = {}
exec_circuits_phase2 = {}
exec_circuits_phase3 = {}
lambda_refs = {}

for i in range(n_samples):
    print(f"Creating circuits for sample {i}/{n_samples}")
    Js = all_Js[i]

    # lambda_ref はこの後も使うので、辞書として保存
    lambda_refs[f"sample{i}"] = np.sum(Js)  # Reference eigenvalue.

    heisenberg_sim = HeisenbergModel(n_qubits, graphs[i], backend=backend_sim)

    circuits_phase0[f"sample{i}"] = {}
    circuits_phase1[f"sample{i}"] = {}
    circuits_phase2[f"sample{i}"] = {}
    circuits_phase3[f"sample{i}"] = {}
    exec_circuits_phase0[f"sample{i}"] = {}
    exec_circuits_phase1[f"sample{i}"] = {}
    exec_circuits_phase2[f"sample{i}"] = {}
    exec_circuits_phase3[f"sample{i}"] = {}
    for k in range(n_features):
        n_steps = get_n_steps(times[k])
        circuit_phase0, exec_circuit_phase0 = (
            heisenberg_sim.get_trotter_simulation_circuit(times[k], n_steps, phase=0)
        )
        circuit_phase1, exec_circuit_phase1 = (
            heisenberg_sim.get_trotter_simulation_circuit(times[k], n_steps, phase=1)
        )
        circuit_phase2, exec_circuit_phase2 = (
            heisenberg_sim.get_trotter_simulation_circuit(times[k], n_steps, phase=2)
        )
        circuit_phase3, exec_circuit_phase3 = (
            heisenberg_sim.get_trotter_simulation_circuit(times[k], n_steps, phase=3)
        )

        circuits_phase0[f"sample{i}"][f"f_{k}"] = circuit_phase0
        circuits_phase1[f"sample{i}"][f"f_{k}"] = circuit_phase1
        circuits_phase2[f"sample{i}"][f"f_{k}"] = circuit_phase2
        circuits_phase3[f"sample{i}"][f"f_{k}"] = circuit_phase3
        exec_circuits_phase0[f"sample{i}"][f"f_{k}"] = exec_circuit_phase0
        exec_circuits_phase1[f"sample{i}"][f"f_{k}"] = exec_circuit_phase1
        exec_circuits_phase2[f"sample{i}"][f"f_{k}"] = exec_circuit_phase2
        exec_circuits_phase3[f"sample{i}"][f"f_{k}"] = exec_circuit_phase3

Creating circuits for sample 0/70
Creating circuits for sample 1/70
Creating circuits for sample 2/70
Creating circuits for sample 3/70
Creating circuits for sample 4/70
Creating circuits for sample 5/70
Creating circuits for sample 6/70
Creating circuits for sample 7/70
Creating circuits for sample 8/70
Creating circuits for sample 9/70
Creating circuits for sample 10/70
Creating circuits for sample 11/70
Creating circuits for sample 12/70
Creating circuits for sample 13/70
Creating circuits for sample 14/70
Creating circuits for sample 15/70
Creating circuits for sample 16/70
Creating circuits for sample 17/70
Creating circuits for sample 18/70
Creating circuits for sample 19/70
Creating circuits for sample 20/70
Creating circuits for sample 21/70
Creating circuits for sample 22/70
Creating circuits for sample 23/70
Creating circuits for sample 24/70
Creating circuits for sample 25/70
Creating circuits for sample 26/70
Creating circuits for sample 27/70
Creating circuits for sample 2

In [9]:
# Run jobs in batch

# For AerSimulator, we can't use job ids. Instead, we store the jobs in a list.
jobs = []
with Batch(backend=backend_sim):
    sampler = Sampler()

    for i in range(n_samples):
        print(f"Running circuits for sample {i}/{n_samples}")
        exec_circuits_per_sample = []
        exec_circuits_per_sample += [
            exec_circuits_phase0[f"sample{i}"][f"f_{k}"] for k in range(n_features)
        ]
        exec_circuits_per_sample += [
            exec_circuits_phase1[f"sample{i}"][f"f_{k}"] for k in range(n_features)
        ]
        exec_circuits_per_sample += [
            exec_circuits_phase2[f"sample{i}"][f"f_{k}"] for k in range(n_features)
        ]
        exec_circuits_per_sample += [
            exec_circuits_phase3[f"sample{i}"][f"f_{k}"] for k in range(n_features)
        ]

        job = sampler.run(exec_circuits_per_sample)
        jobs.append(job)

Running circuits for sample 0/70
Running circuits for sample 1/70
Running circuits for sample 2/70
Running circuits for sample 3/70
Running circuits for sample 4/70
Running circuits for sample 5/70
Running circuits for sample 6/70
Running circuits for sample 7/70
Running circuits for sample 8/70
Running circuits for sample 9/70
Running circuits for sample 10/70
Running circuits for sample 11/70
Running circuits for sample 12/70
Running circuits for sample 13/70
Running circuits for sample 14/70
Running circuits for sample 15/70
Running circuits for sample 16/70
Running circuits for sample 17/70
Running circuits for sample 18/70
Running circuits for sample 19/70
Running circuits for sample 20/70
Running circuits for sample 21/70
Running circuits for sample 22/70
Running circuits for sample 23/70
Running circuits for sample 24/70
Running circuits for sample 25/70
Running circuits for sample 26/70
Running circuits for sample 27/70
Running circuits for sample 28/70
Running circuits for sam

In [10]:
check_list = []
for i in range(n_samples):
    job_id = jobs[i].job_id()
    check_list.append(
        {
            "Job ID": job_id,
            "sample_id": i,
            "Status": job.status().name,
        }
    )

check_df = pd.DataFrame(check_list)
display(check_df)

,Job ID,sample_id,Status
0,367f67bf-ea14-42fe-9cd8-8b1a8de98a6b,0,RUNNING
1,c7ba2126-c8f8-45c1-923a-75a804b08421,1,RUNNING
2,70e3560c-54c1-4db2-a24e-53016eef01ec,2,RUNNING
3,860e9d4c-0eba-4194-b482-9e41aaeb4460,3,RUNNING
4,5ce8aee3-ba58-4fa0-9eb2-b50ea26341ce,4,RUNNING
...,...,...,...
65,705582a6-db17-4e56-884d-cf025878cc23,65,RUNNING
66,6144bbd5-7173-41e8-9904-503937c5783f,66,RUNNING
67,3878ea10-e10a-4646-98f7-af6dce0f612c,67,RUNNING
68,590d9e29-b8c5-4a82-90a3-e26f1ba0a349,68,RUNNING


In [11]:
# Post-process
data_sim = []
probs_phase0_sim = {}
probs_phase1_sim = {}
probs_phase2_sim = {}
probs_phase3_sim = {}

for i in range(n_samples):
    features = []
    probs_phase0_sim[f"sample{i}"] = {}
    probs_phase1_sim[f"sample{i}"] = {}
    probs_phase2_sim[f"sample{i}"] = {}
    probs_phase3_sim[f"sample{i}"] = {}

    # Compute the Fourier features for different times
    for k in range(n_features):
        # Get results of each phase in a batch
        results_phase0 = jobs[i].result()[:n_features]
        results_phase1 = jobs[i].result()[n_features : 2 * n_features]
        results_phase2 = jobs[i].result()[2 * n_features : 3 * n_features]
        results_phase3 = jobs[i].result()[3 * n_features :]

        prob_phase0 = get_prob0(results_phase0[k], n_qubits)
        prob_phase1 = get_prob0(results_phase1[k], n_qubits)
        prob_phase2 = get_prob0(results_phase2[k], n_qubits)
        prob_phase3 = get_prob0(results_phase3[k], n_qubits)

        probs_phase0_sim[f"sample{i}"][f"f_{k}"] = prob_phase0
        probs_phase1_sim[f"sample{i}"][f"f_{k}"] = prob_phase1
        probs_phase2_sim[f"sample{i}"][f"f_{k}"] = prob_phase2
        probs_phase3_sim[f"sample{i}"][f"f_{k}"] = prob_phase3

        inner_product = np.exp(-1j * lambda_refs[f"sample{i}"] * times[k]) * (
            (prob_phase0 - prob_phase1) + 1j * (prob_phase2 - prob_phase3)
        )

        features.append(inner_product.real)
        if k != 0:
            features.append(inner_product.imag)
    data_sim.append([i, *features, all_expected_values[i]])

# Create column names for the DataFrame
columns = []
columns.append("sample_id")
for k in range(n_features):
    columns.append(f"f_{k} Re")
    if k != 0:
        columns.append(f"f_{k} Im")
columns.append("expected_value")

# Convert to a DataFrame
data_sim_df = pd.DataFrame(data_sim, columns=columns)
display(data_sim_df)

# Save the simulation data
data_sim_df.to_json(f"{path}/data_sim_df.json")
with open(f"{path}/probs_phase0_sim.json", "w") as f:
    json.dump(probs_phase0_sim, f)

with open(f"{path}/probs_phase1_sim.json", "w") as f:
    json.dump(probs_phase1_sim, f)

with open(f"{path}/probs_phase2_sim.json", "w") as f:
    json.dump(probs_phase2_sim, f)

with open(f"{path}/probs_phase3_sim.json", "w") as f:
    json.dump(probs_phase3_sim, f)

 > No counts for |0...0> state
 > No counts for |0...0> state
 > No counts for |0...0> state
 > No counts for |0...0> state
 > No counts for |0...0> state
 > No counts for |0...0> state
 > No counts for |0...0> state
 > No counts for |0...0> state
 > No counts for |0...0> state
 > No counts for |0...0> state
 > No counts for |0...0> state
 > No counts for |0...0> state
 > No counts for |0...0> state
 > No counts for |0...0> state
 > No counts for |0...0> state
 > No counts for |0...0> state
 > No counts for |0...0> state
 > No counts for |0...0> state
 > No counts for |0...0> state
 > No counts for |0...0> state
 > No counts for |0...0> state
 > No counts for |0...0> state
 > No counts for |0...0> state
 > No counts for |0...0> state
 > No counts for |0...0> state
 > No counts for |0...0> state
 > No counts for |0...0> state
 > No counts for |0...0> state
 > No counts for |0...0> state
 > No counts for |0...0> state
 > No counts for |0...0> state
 > No counts for |0...0> state
 > No co

,sample_id,f_0 Re,f_1 Re,f_1 Im,f_2 Re,f_2 Im,f_3 Re,f_3 Im,f_4 Re,f_4 Im,...,f_5 Im,f_6 Re,f_6 Im,f_7 Re,f_7 Im,f_8 Re,f_8 Im,f_9 Re,f_9 Im,expected_value
0,0,1.0,0.812447,-0.142416,0.434590,-0.110929,0.131897,0.001402,-0.148986,0.115820,...,0.162113,-0.422264,0.113535,-0.369654,0.066869,-0.275181,-0.140577,-0.200590,-0.294542,0.978582
1,1,1.0,0.679222,-0.555266,0.091608,-0.589609,-0.282322,-0.132047,-0.053795,0.276423,...,0.151757,0.438162,-0.315640,0.045548,-0.682327,-0.436457,-0.581458,-0.655494,-0.127316,0.904499
2,2,1.0,0.925269,-0.130632,0.709199,-0.221466,0.447844,-0.281728,0.139920,-0.168262,...,0.121536,-0.094452,0.340240,-0.046472,0.576712,0.099466,0.700316,0.221143,0.745497,0.974517
3,3,1.0,0.791672,0.340737,0.403211,0.293373,0.236246,0.035276,0.318808,-0.055728,...,0.074770,-0.058835,0.002632,-0.188605,-0.309996,0.079994,-0.537765,0.349060,-0.409607,1.070568
4,4,1.0,0.963321,-0.060926,0.864568,-0.073660,0.711590,-0.094475,0.576764,-0.053994,...,-0.080880,0.397815,-0.051348,0.285199,-0.039783,0.251774,-0.111582,0.228715,-0.099156,0.990622
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65,65,1.0,0.687462,0.426793,0.076662,0.471600,-0.354042,0.035286,-0.206494,-0.543173,...,-0.680316,0.775284,-0.354484,0.735545,0.150674,0.263083,0.337864,-0.165298,0.101254,1.088310
66,66,1.0,0.822855,-0.166201,0.454665,-0.144146,0.145039,0.083224,-0.087261,0.198882,...,0.295649,-0.049176,0.360812,0.075031,0.328221,0.197446,0.264541,0.132841,0.124453,0.977840
67,67,1.0,0.890243,0.377886,0.567325,0.605302,0.182723,0.728968,-0.184863,0.615789,...,0.335224,-0.515405,0.132448,-0.432810,-0.161910,-0.260201,-0.321400,-0.043951,-0.265890,1.064102
68,68,1.0,0.922838,-0.038015,0.744613,-0.097326,0.490809,-0.136274,0.141541,-0.013641,...,0.034858,-0.232483,0.180429,-0.290724,0.295677,-0.207698,0.380351,-0.091868,0.369114,0.988139


## Trotter simulation (Simulator, Noisy)

In [12]:
# Create circuits
# 1 jobs - 1 sample
circuits_phase0 = {}
circuits_phase1 = {}
circuits_phase2 = {}
circuits_phase3 = {}
exec_circuits_phase0 = {}
exec_circuits_phase1 = {}
exec_circuits_phase2 = {}
exec_circuits_phase3 = {}

for i in range(n_samples):
    print(f"Creating circuits for sample {i}/{n_samples}")
    Js = all_Js[i]
    heisenberg_sim_noisy = HeisenbergModel(
        n_qubits, graphs[i], backend=backend_sim_noisy
    )

    circuits_phase0[f"sample{i}"] = {}
    circuits_phase1[f"sample{i}"] = {}
    circuits_phase2[f"sample{i}"] = {}
    circuits_phase3[f"sample{i}"] = {}
    exec_circuits_phase0[f"sample{i}"] = {}
    exec_circuits_phase1[f"sample{i}"] = {}
    exec_circuits_phase2[f"sample{i}"] = {}
    exec_circuits_phase3[f"sample{i}"] = {}
    for k in range(n_features):
        n_steps = get_n_steps(times[k])
        circuit_phase0, exec_circuit_phase0 = (
            heisenberg_sim_noisy.get_trotter_simulation_circuit(
                times[k], n_steps, phase=0
            )
        )
        circuit_phase1, exec_circuit_phase1 = (
            heisenberg_sim_noisy.get_trotter_simulation_circuit(
                times[k], n_steps, phase=1
            )
        )
        circuit_phase2, exec_circuit_phase2 = (
            heisenberg_sim_noisy.get_trotter_simulation_circuit(
                times[k], n_steps, phase=2
            )
        )
        circuit_phase3, exec_circuit_phase3 = (
            heisenberg_sim_noisy.get_trotter_simulation_circuit(
                times[k], n_steps, phase=3
            )
        )

        circuits_phase0[f"sample{i}"][f"f_{k}"] = circuit_phase0
        circuits_phase1[f"sample{i}"][f"f_{k}"] = circuit_phase1
        circuits_phase2[f"sample{i}"][f"f_{k}"] = circuit_phase2
        circuits_phase3[f"sample{i}"][f"f_{k}"] = circuit_phase3
        exec_circuits_phase0[f"sample{i}"][f"f_{k}"] = exec_circuit_phase0
        exec_circuits_phase1[f"sample{i}"][f"f_{k}"] = exec_circuit_phase1
        exec_circuits_phase2[f"sample{i}"][f"f_{k}"] = exec_circuit_phase2
        exec_circuits_phase3[f"sample{i}"][f"f_{k}"] = exec_circuit_phase3

Creating circuits for sample 0/70
Creating circuits for sample 1/70
Creating circuits for sample 2/70
Creating circuits for sample 3/70
Creating circuits for sample 4/70
Creating circuits for sample 5/70
Creating circuits for sample 6/70
Creating circuits for sample 7/70
Creating circuits for sample 8/70
Creating circuits for sample 9/70
Creating circuits for sample 10/70
Creating circuits for sample 11/70
Creating circuits for sample 12/70
Creating circuits for sample 13/70
Creating circuits for sample 14/70
Creating circuits for sample 15/70
Creating circuits for sample 16/70
Creating circuits for sample 17/70
Creating circuits for sample 18/70
Creating circuits for sample 19/70
Creating circuits for sample 20/70
Creating circuits for sample 21/70
Creating circuits for sample 22/70
Creating circuits for sample 23/70
Creating circuits for sample 24/70
Creating circuits for sample 25/70
Creating circuits for sample 26/70
Creating circuits for sample 27/70
Creating circuits for sample 2

In [13]:
# Run jobs in batch
jobs = []  # For AerSimulator, we can't use job ids.
with Batch(backend=backend_sim_noisy):
    sampler = Sampler()

    for i in range(n_samples):
        print(f"Running circuits for sample {i}/{n_samples}")
        exec_circuits_per_sample = []
        exec_circuits_per_sample += [
            exec_circuits_phase0[f"sample{i}"][f"f_{k}"] for k in range(n_features)
        ]
        exec_circuits_per_sample += [
            exec_circuits_phase1[f"sample{i}"][f"f_{k}"] for k in range(n_features)
        ]
        exec_circuits_per_sample += [
            exec_circuits_phase2[f"sample{i}"][f"f_{k}"] for k in range(n_features)
        ]
        exec_circuits_per_sample += [
            exec_circuits_phase3[f"sample{i}"][f"f_{k}"] for k in range(n_features)
        ]

        job = sampler.run(exec_circuits_per_sample)
        jobs.append(job)

Running circuits for sample 0/70
Running circuits for sample 1/70
Running circuits for sample 2/70
Running circuits for sample 3/70
Running circuits for sample 4/70
Running circuits for sample 5/70
Running circuits for sample 6/70
Running circuits for sample 7/70
Running circuits for sample 8/70
Running circuits for sample 9/70
Running circuits for sample 10/70
Running circuits for sample 11/70
Running circuits for sample 12/70
Running circuits for sample 13/70
Running circuits for sample 14/70
Running circuits for sample 15/70
Running circuits for sample 16/70
Running circuits for sample 17/70
Running circuits for sample 18/70
Running circuits for sample 19/70
Running circuits for sample 20/70
Running circuits for sample 21/70
Running circuits for sample 22/70
Running circuits for sample 23/70
Running circuits for sample 24/70
Running circuits for sample 25/70
Running circuits for sample 26/70
Running circuits for sample 27/70
Running circuits for sample 28/70
Running circuits for sam

In [14]:
check_list = []
for i in range(n_samples):
    job_id = jobs[i].job_id()
    check_list.append(
        {
            "Job ID": job_id,
            "sample_id": i,
            "Status": job.status().name,
        }
    )

check_df = pd.DataFrame(check_list)
display(check_df)

,Job ID,sample_id,Status
0,b6f0e5bd-cfdd-46ce-af13-2edf8a52971c,0,RUNNING
1,a2b5a903-bb72-46d5-99d5-46a9c54e57e7,1,RUNNING
2,b6a63024-219b-4301-b8e3-dbebe7e1617c,2,RUNNING
3,ea402e18-c6ee-423f-9d1c-c9fb33cc2aec,3,RUNNING
4,0ddbbe04-7d3d-4101-8e15-d5138d473704,4,RUNNING
...,...,...,...
65,11ffb47a-48a1-4dcd-802c-66d36c805996,65,RUNNING
66,6357e464-fda7-4321-b67e-dde5f996a186,66,RUNNING
67,7959c8f0-779c-425c-8225-7a08803d8000,67,RUNNING
68,246a34d5-67d1-4b8b-a8e0-18e0b2f3c8e8,68,RUNNING


In [ ]:
# Post-process
data_sim_noisy = []
probs_phase0_sim_noisy = {}
probs_phase1_sim_noisy = {}
probs_phase2_sim_noisy = {}
probs_phase3_sim_noisy = {}

for i in range(n_samples):
    features = []
    probs_phase0_sim_noisy[f"sample{i}"] = {}
    probs_phase1_sim_noisy[f"sample{i}"] = {}
    probs_phase2_sim_noisy[f"sample{i}"] = {}
    probs_phase3_sim_noisy[f"sample{i}"] = {}

    # Compute the Fourier features for different times
    for k in range(n_features):
        # Get results of each phase in a batch
        results_phase0 = jobs[i].result()[:n_features]
        results_phase1 = jobs[i].result()[n_features : 2 * n_features]
        results_phase2 = jobs[i].result()[2 * n_features : 3 * n_features]
        results_phase3 = jobs[i].result()[3 * n_features :]

        prob_phase0 = get_prob0(results_phase0[k], n_qubits)
        prob_phase1 = get_prob0(results_phase1[k], n_qubits)
        prob_phase2 = get_prob0(results_phase2[k], n_qubits)
        prob_phase3 = get_prob0(results_phase3[k], n_qubits)

        probs_phase0_sim_noisy[f"sample{i}"][f"f_{k}"] = prob_phase0
        probs_phase1_sim_noisy[f"sample{i}"][f"f_{k}"] = prob_phase1
        probs_phase2_sim_noisy[f"sample{i}"][f"f_{k}"] = prob_phase2
        probs_phase3_sim_noisy[f"sample{i}"][f"f_{k}"] = prob_phase3

        inner_product = np.exp(-1j * lambda_refs[f"sample{i}"] * times[k]) * (
            (prob_phase0 - prob_phase1) + 1j * (prob_phase2 - prob_phase3)
        )
        features.append(inner_product.real)
        if k != 0:
            features.append(inner_product.imag)
    data_sim_noisy.append([i, *features, all_expected_values[i]])

# Create column names for the DataFrame
columns = []
columns.append("sample_id")
for k in range(n_features):
    columns.append(f"f_{k} Re")
    if k != 0:
        columns.append(f"f_{k} Im")
columns.append("expected_value")

# Convert to a DataFrame
data_sim_noisy_df = pd.DataFrame(data_sim_noisy, columns=columns)
display(data_sim_noisy_df)

# Save the simulation data
data_sim_noisy_df.to_json(f"{path}/data_sim_noisy_df.json")
with open(f"{path}/probs_phase0_sim_noisy.json", "w") as f:
    json.dump(probs_phase0_sim_noisy, f)

with open(f"{path}/probs_phase1_sim_noisy.json", "w") as f:
    json.dump(probs_phase1_sim_noisy, f)

with open(f"{path}/probs_phase2_sim_noisy.json", "w") as f:
    json.dump(probs_phase2_sim_noisy, f)

with open(f"{path}/probs_phase3_sim_noisy.json", "w") as f:
    json.dump(probs_phase3_sim_noisy, f)

In [ ]:
print("Finished!")